In [4]:
import pdfplumber
import pandas as pd
import re

pdf_path = "D:/pdftoexcel/data/pre_monsoon_1994_2003_split/pre_monsoon_1994_2003-2001-3047.pdf"

output_csv_path = r"D:/pdftoexcel/data/pre_monsoon_1994_2003_split/csv_dataset/extracted_data_2001_3047.csv"

extracted_data = []

# Using a slightly more robust regex to handle different number formats
line_pattern = re.compile(r'(\d+\.?\d*)\s+(\d+\.?\d*)\s+(\d{2}-\d{2}-\d{2})\s+([\d\.]+)$')

# --- FIX: Define line_buffer OUTSIDE the page loop ---
line_buffer = ""

with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue

        raw_lines = text.split('\n')
        processed_lines = []
        
        for line in raw_lines:
            line_buffer += " " + line.strip()
            if line_pattern.search(line_buffer):
                processed_lines.append(line_buffer.strip())
                line_buffer = "" # Reset buffer ONLY after a successful record is found

        for line in processed_lines:
            match = line_pattern.search(line)
            if match:
                latitude, longitude, date, wl = match.groups()
                location_info = line[:match.start()].strip()
                
                extracted_data.append({
                    'Location_Info': location_info,
                    'Latitude': latitude,
                    'Longitude': longitude,
                    'Date': date,
                    'WL(mbgl)': wl
                })

if extracted_data:
    df = pd.DataFrame(extracted_data)
    df.to_csv(output_csv_path, index=False)